In [1]:
val aisles = spark.read.format("csv")
  .option("sep", ",")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("aisles.csv")

Intitializing Scala interpreter ...

Spark Web UI available at http://91ad3f587c27:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1616136030165)
SparkSession available as 'spark'


aisles: org.apache.spark.sql.DataFrame = [aisle_id: int, aisle: string]


In [2]:
sc.version

res0: String = 3.1.1


In [3]:
val departments = spark.read.format("csv")
  .option("sep", ",")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("departments.csv")

departments: org.apache.spark.sql.DataFrame = [department_id: int, department: string]


In [4]:
val order_products_prior = spark.read.format("csv")
  .option("sep", ",")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("order_products__prior.csv")

order_products_prior: org.apache.spark.sql.DataFrame = [order_id: int, product_id: int ... 2 more fields]


In [5]:
val order_products_train = spark.read.format("csv")
  .option("sep", ",")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("order_products__train.csv")

order_products_train: org.apache.spark.sql.DataFrame = [order_id: int, product_id: int ... 2 more fields]


In [6]:
aisles.count()

res1: Long = 134


In [7]:
departments.count()

res2: Long = 21


In [8]:
order_products_prior.count()

res3: Long = 32434489


In [9]:
val orders = spark.read.format("csv")
  .option("sep", ",")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("orders.csv")

orders: org.apache.spark.sql.DataFrame = [order_id: int, user_id: int ... 5 more fields]


In [10]:
val products = spark.read.format("csv")
  .option("sep", ",")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("products.csv")

products: org.apache.spark.sql.DataFrame = [product_id: int, product_name: string ... 2 more fields]


In [11]:
import org.apache.spark.sql._
import org.apache.log4j._
import org.apache.spark.sql.types.{StructType, StructField, StringType}
import org.apache.spark.sql.Row

val sqlContext = new SQLContext(sc)

import org.apache.spark.sql._
import org.apache.log4j._
import org.apache.spark.sql.types.{StructType, StructField, StringType}
import org.apache.spark.sql.Row
sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@226dbb86


In [12]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- eval_set: string (nullable = true)
 |-- order_number: integer (nullable = true)
 |-- order_dow: integer (nullable = true)
 |-- order_hour_of_day: integer (nullable = true)
 |-- days_since_prior_order: double (nullable = true)



In [13]:
orders.registerTempTable("orders")

In [14]:
val allrecords = sqlContext.sql("select * from orders")

allrecords: org.apache.spark.sql.DataFrame = [order_id: int, user_id: int ... 5 more fields]


In [15]:
allrecords.show()

+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
| 2539329|      1|   prior|           1|        2|                8|                  null|
| 2398795|      1|   prior|           2|        3|                7|                  15.0|
|  473747|      1|   prior|           3|        3|               12|                  21.0|
| 2254736|      1|   prior|           4|        4|                7|                  29.0|
|  431534|      1|   prior|           5|        4|               15|                  28.0|
| 3367565|      1|   prior|           6|        2|                7|                  19.0|
|  550135|      1|   prior|           7|        1|                9|                  20.0|
| 3108588|      1|   prior|           8|        1|               14|            

In [16]:
val total_orders = sqlContext.sql("""
| select 
|  count(order_id) as total_orders, 
|  (case 
|     when order_dow = '0' then 'Sunday'
|     when order_dow = '1' then 'Monday'
|     when order_dow = '2' then 'Tuesday'
|     when order_dow = '3' then 'Wednesday'
|     when order_dow = '4' then 'Thursday'
|     when order_dow = '5' then 'Friday'
|     when order_dow = '6' then 'Saturday'              
|   end) as day_of_week 
|  from orders
| group by order_dow 
| order by total_orders desc
 """)

total_orders: org.apache.spark.sql.DataFrame = [total_orders: bigint, day_of_week: string]


In [17]:
total_orders.show()
  

+------------+-----------+
|total_orders|day_of_week|
+------------+-----------+
|      600905|     Sunday|
|      587478|     Monday|
|      467260|    Tuesday|
|      453368|     Friday|
|      448761|   Saturday|
|      436972|  Wednesday|
|      426339|   Thursday|
+------------+-----------+



In [18]:
val total_orders_by_hour = sqlContext.sql("""
|select 
|  count(order_id) as total_orders, 
|  order_hour_of_day as hour 
|  from orders 
| group by order_hour_of_day 
| order by order_hour_of_day
 """)

total_orders_by_hour: org.apache.spark.sql.DataFrame = [total_orders: bigint, hour: int]


In [19]:
total_orders_by_hour.show()

+------------+----+
|total_orders|hour|
+------------+----+
|       22758|   0|
|       12398|   1|
|        7539|   2|
|        5474|   3|
|        5527|   4|
|        9569|   5|
|       30529|   6|
|       91868|   7|
|      178201|   8|
|      257812|   9|
|      288418|  10|
|      284728|  11|
|      272841|  12|
|      277999|  13|
|      283042|  14|
|      283639|  15|
|      272553|  16|
|      228795|  17|
|      182912|  18|
|      140569|  19|
+------------+----+
only showing top 20 rows



In [27]:
products.registerTempTable("products")
order_products_train.registerTempTable("order_products_train")
val rawData = sqlContext.sql("select p.product_name, o.order_id from products p inner join order_products_train o where o.product_id = p.product_id")


rawData: org.apache.spark.sql.DataFrame = [product_name: string, order_id: int]


In [29]:
val baskets = rawData.groupBy("order_id").agg(collect_set("product_name").alias("items"))
baskets.createOrReplaceTempView("baskets")

baskets: org.apache.spark.sql.DataFrame = [order_id: int, items: array<string>]


In [30]:
import org.apache.spark.ml.fpm.FPGrowth

// Extract out the items 
val baskets_ds = spark.sql("select items from baskets").as[Array[String]].toDF("items")

// Use FPGrowth
val fpgrowth = new FPGrowth().setItemsCol("items").setMinSupport(0.001).setMinConfidence(0)
val model = fpgrowth.fit(baskets_ds)

// Calculate frequent itemsets
val mostPopularItemInABasket = model.freqItemsets
mostPopularItemInABasket.createOrReplaceTempView("mostPopularItemInABasket")

import org.apache.spark.ml.fpm.FPGrowth
baskets_ds: org.apache.spark.sql.DataFrame = [items: array<string>]
fpgrowth: org.apache.spark.ml.fpm.FPGrowth = fpgrowth_d8374f5832ce
model: org.apache.spark.ml.fpm.FPGrowthModel = FPGrowthModel: uid=fpgrowth_d8374f5832ce, numTrainingRecords=131209
mostPopularItemInABasket: org.apache.spark.sql.DataFrame = [items: array<string>, freq: bigint]


In [34]:
val top_most = spark.sql("select items, freq from mostPopularItemInABasket where size(items) > 2 order by freq desc limit 20")

top_most: org.apache.spark.sql.DataFrame = [items: array<string>, freq: bigint]


In [35]:
top_most.show(false)

+--------------------------------------------------------------------+----+
|items                                                               |freq|
+--------------------------------------------------------------------+----+
|[Organic Hass Avocado, Organic Strawberries, Bag of Organic Bananas]|710 |
|[Organic Raspberries, Organic Strawberries, Bag of Organic Bananas] |649 |
|[Organic Baby Spinach, Organic Strawberries, Bag of Organic Bananas]|587 |
|[Organic Raspberries, Organic Hass Avocado, Bag of Organic Bananas] |531 |
|[Organic Hass Avocado, Organic Baby Spinach, Bag of Organic Bananas]|497 |
|[Organic Avocado, Organic Baby Spinach, Banana]                     |484 |
|[Organic Avocado, Large Lemon, Banana]                              |477 |
|[Limes, Large Lemon, Banana]                                        |452 |
|[Organic Cucumber, Organic Strawberries, Bag of Organic Bananas]    |424 |
|[Limes, Organic Avocado, Large Lemon]                               |389 |
|[Organic Ra

In [36]:
// Display generated association rules.
val ifThen = model.associationRules
ifThen.createOrReplaceTempView("ifThen")

ifThen: org.apache.spark.sql.DataFrame = [antecedent: array<string>, consequent: array<string> ... 3 more fields]


In [37]:
val rules = spark.sql("select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence from ifThen order by confidence desc limit 20")

rules: org.apache.spark.sql.DataFrame = [antecedent (if): array<string>, consequent (then): array<string> ... 1 more field]


In [39]:
rules.show(false)

+------------------------------------------------------------------+------------------------+-------------------+
|antecedent (if)                                                   |consequent (then)       |confidence         |
+------------------------------------------------------------------+------------------------+-------------------+
|[Organic Raspberries, Organic Hass Avocado, Organic Strawberries] |[Bag of Organic Bananas]|0.5984251968503937 |
|[Organic Cucumber, Organic Hass Avocado, Organic Strawberries]    |[Bag of Organic Bananas]|0.546875           |
|[Organic Kiwi, Organic Hass Avocado]                              |[Bag of Organic Bananas]|0.5459770114942529 |
|[Organic Navel Orange, Organic Raspberries]                       |[Bag of Organic Bananas]|0.5412186379928315 |
|[Yellow Onions, Strawberries]                                     |[Banana]                |0.5357142857142857 |
|[Organic Whole String Cheese, Organic Hass Avocado]               |[Bag of Organic Bana